In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
model_path = "/content/drive/MyDrive/job_question_generator3"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [8]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()
if torch.cuda.is_available():
    model.to("cuda")

def generate_questions(job_title, num_questions=5):
    prompt = f"Role: {job_title}\nQuestion:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_new_tokens=50,
            num_return_sequences=num_questions,
            do_sample=True,
            temperature=0.8,
            top_p=0.9
        )

    questions = []
    for o in outputs:
        text = tokenizer.decode(o, skip_special_tokens=True)
        question_only = text.replace(f"Role: {job_title}\nQuestion:", "").strip()
        questions.append(question_only)

    return "\n".join(questions)

iface = gr.Interface(
    fn=generate_questions,
    inputs=[gr.Textbox(label="Job Title")],
    outputs=gr.Textbox(label="Generated Questions", lines=10),
    title="Interview Question Generator"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ede37caaab3b6fcc4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
